<a href="https://colab.research.google.com/github/RohanGforwork/ML_text_summarization/blob/main/team_1_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00


In [5]:
df = pd.read_csv('/content/drive/MyDrive/datasets_foss/twcs.csv')
df

KeyboardInterrupt: 

In [31]:
from datasets import load_dataset

df = load_dataset("agentlans/wikipedia-paragraph-summaries")

In [32]:
from datasets import load_dataset
import pandas as pd

# Load dataset
dataset = load_dataset("agentlans/wikipedia-paragraph-summaries")

# Convert to Pandas DataFrame (for 'train' split)
df = dataset["train"].to_pandas()

# Show first few rows
print(df.head())


   id                                              input  \
0   1  Perhaps the first public statement on the matt...   
1   2  Over the next several decades, the death penal...   
2   3  Chester Himes was born in Jefferson City, Miss...   
3   4  The Euallomyces life cycle is an anisogamous a...   
4   5  Humenik joined the PGA Tour in 1989, gaining h...   

                                              output  
0  Catherine II's 1767 statement and later events...  
1  The Soviet death penalty had a complex history...  
2  Chester Himes, born in Missouri in 1909, grew ...  
3  The Euallomyces life cycle alternates between ...  
4  Humenik began his PGA Tour career in 1989, str...  


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21811 entries, 0 to 21810
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      21811 non-null  int64 
 1   input   21811 non-null  object
 2   output  21811 non-null  object
dtypes: int64(1), object(2)
memory usage: 511.3+ KB


In [34]:
df.isnull().sum()

,0
id,0
input,0
output,0


In [37]:

df = df['input'].to_frame()

In [38]:
print(df.head())

                                               input
0  Perhaps the first public statement on the matt...
1  Over the next several decades, the death penal...
2  Chester Himes was born in Jefferson City, Miss...
3  The Euallomyces life cycle is an anisogamous a...
4  Humenik joined the PGA Tour in 1989, gaining h...


In [ ]:
'''pip install datasets'''

In [ ]:
import pandas as pd
import spacy
import re
import string
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load dataset from Hugging Face
dataset = load_dataset("agentlans/wikipedia-paragraph-summaries")

# Convert to Pandas DataFrame
df = dataset["train"].to_pandas()

# Load English NLP model (disable unnecessary components for speed)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Precompile regex and translation table for faster processing
url_pattern = re.compile(r"http\S+")
translator = str.maketrans("", "", string.punctuation)

def clean_text(text):
    """Basic text cleaning: Lowercase, remove URLs & punctuation"""
    text = text.lower()
    text = url_pattern.sub("", text)  # Remove URLs
    text = text.translate(translator)  # Remove punctuation
    return text

# Apply initial cleaning to paragraphs & summaries
df["cleaned_paragraph"] = df["input"].apply(clean_text)
df["cleaned_summary"] = df["output"].apply(clean_text)

def process_texts(texts):
    """Lemmatization & stopword removal using spaCy (batch processing)"""
    docs = list(nlp.pipe(texts, batch_size=512))  # Adjust batch_size based on RAM
    return [" ".join([token.lemma_ for token in doc if not token.is_stop]) for doc in docs]

# Apply NLP processing in batches
df["cleaned_paragraph"] = process_texts(df["cleaned_paragraph"])
df["cleaned_summary"] = process_texts(df["cleaned_summary"])

# Train-Test Split
train_paragraphs, test_paragraphs, train_summaries, test_summaries = train_test_split(
    df["cleaned_paragraph"], df["cleaned_summary"], test_size=0.2, random_state=42
)

print("✅ Preprocessing Completed!")


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-local-base")

# Tokenize training and testing data
train_encodings = tokenizer(list(train_texts), padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(list(test_texts), padding=True, truncation=True, max_length=512, return_tensors="pt")


In [ ]:
# long t5 conditional generation
from transformers import LongT5ForConditionalGeneration, Trainer, TrainingArguments
import torch

model = LongT5ForConditionalGeneration.from_pretrained("google/long-t5-local-base")

# Convert data to PyTorch tensors
train_labels = tokenizer(list(train_texts), padding=True, truncation=True, max_length=150, return_tensors="pt").input_ids
test_labels = tokenizer(list(test_texts), padding=True, truncation=True, max_length=150, return_tensors="pt").input_ids

train_data = [{"input_ids": train_encodings.input_ids[i], "attention_mask": train_encodings.attention_mask[i], "labels": train_labels[i]} for i in range(len(train_texts))]
test_data = [{"input_ids": test_encodings.input_ids[i], "attention_mask": test_encodings.attention_mask[i], "labels": test_labels[i]} for i in range(len(test_texts))]

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Start Training
trainer.train()


In [ ]:
model.save_pretrained("./fine_tuned_longt5")
tokenizer.save_pretrained("./fine_tuned_longt5")


In [ ]:
def summarize_tweet(tweet, model, tokenizer):
    input_ids = tokenizer.encode("summarize: " + tweet, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=3, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

test_tweet = "Your sample social media text here..."
print("Summary:", summarize_tweet(test_tweet, model, tokenizer))


In [ ]:
import spacy
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load Summarization Models
pegasus_model_name = "google/pegasus-xsum"
longt5_model_name = "google/long-t5-tglobal-base"

pegasus_tokenizer = AutoTokenizer.from_pretrained(pegasus_model_name)
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(pegasus_model_name)

longt5_tokenizer = AutoTokenizer.from_pretrained(longt5_model_name)
longt5_model = AutoModelForSeq2SeqLM.from_pretrained(longt5_model_name)

# Function to clean text using spaCy
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation

    doc = nlp(text)  # Process text with spaCy
    cleaned_text = " ".join([token.lemma_ for token in doc if not token.is_stop])  # Lemmatize & remove stopwords
    return cleaned_text

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/datasets_foss/twcs.csv')

# Apply text cleaning
df['cleaned_text'] = df['text'].apply(clean_text)

# Convert text to TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limit vocabulary size for efficiency
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

# Function to summarize text dynamically
def summarize_text(text):
    text_length = len(text)

    # Select model dynamically
    if text_length <= 300:
        tokenizer, model = pegasus_tokenizer, pegasus_model
        max_len = 128  # Shorter summary for tweets
    else:
        tokenizer, model = longt5_tokenizer, longt5_model
        max_len = 200  # Longer summary for paragraphs

    # Tokenize input
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    summary_ids = model.generate(input_ids, max_length=max_len, num_beams=5, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply summarization function
df['summary'] = df['cleaned_text'].apply(summarize_text)

# Display results
print(df[['text', 'summary']].head())


In [ ]:
df['text'][142592]

In [ ]:
df

In [ ]:
'''!pip install transformers[torch]
!pip install accelerate -U'''

In [ ]:
import os

In [ ]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer

model = LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

def summarize_tweet(tweet, model=model, tokenizer=tokenizer):
    # Preprocess and tokenize the tweet
    input_ids = tokenizer.encode("summarize: " + tweet, return_tensors="pt", max_length=1000, truncation=True)

    # Generate summary using the LongT5 model
    summary_ids = model.generate(input_ids, max_length=512, num_beams=1, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

tweet = input("Enter a tweet to summarize: ")
summary = summarize_tweet(tweet)

print("Original tweet:", tweet)
print("Summary:", summary)